In [77]:
import numpy as np ; import pandas as pd 
import matplotlib.pyplot as plt 
from help import find_file,list_files
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

### ColumnTranformer 
##### Column tansformer enables us to make transformation to indivisual columns in a single line 

In [48]:
data = pd.read_csv(find_file("adult.csv"))
data = data.drop(columns=["Capital Gain","Capital Loss","Education Num","Hours/Week","fnlgwt"])
data.Race.replace({" Asian-Pac-Islander":"Other"," Amer-Indian-Eskimo":"Other"," Other":"Other"},inplace=True)

data.Workclass.replace({" State-gov":"gov"," Local-gov":"gov"," Federal-gov":"gov",
                     " Never-worked":"unemployed"," Without-pay":"unemployed",
                     " Private":"other"," Self-emp-inc":"other"," Self-emp-not-inc":"other",' ?':"other"},inplace=True)
data  = data.rename(columns={"Above/Below 50k":"Income"})
data.head()

,Age,Workclass,Education,Marital Status,Occupation,Relationship,Race,Sex,Country,Income
0,39,gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K
1,50,other,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K
2,38,other,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K
3,53,other,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K
4,28,other,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K


In [49]:
X  =  data.drop(columns=["Income"],axis=1)
y=  data.Income
x_train,x_test,y_train,y_test =  train_test_split(X,y,test_size=0.2)
x_train.shape,y_train.shape

((26048, 9), (26048,))

In [50]:
column_transformer=  ColumnTransformer(transformers=[
    ('tnf1',OrdinalEncoder(categories=[['Other',' Black',' White']]),['Race']),
    ('tnf2',OneHotEncoder(),['Sex','Workclass']),
    
],remainder="passthrough")

In [51]:
x_train_ct = column_transformer.fit_transform(x_train)

In [52]:
x_train_ct[:1]

array([[2.0, 1.0, 0.0, 0.0, 1.0, 0.0, 33, ' Assoc-voc',
        ' Married-civ-spouse', ' Prof-specialty', ' Wife',
        ' United-States']], dtype=object)

In [53]:
x_train_ct.shape

(26048, 12)

### PipeLine 
##### Pipeline requires naming of steps, make_pipeline does not.

##### (Same applies to ColumnTransformer vs make_column_transformer)

In [59]:
data  =  pd.read_csv(find_file("Social_Network_Ads.csv"))
data =  data.drop(columns=["User ID"])
data.head()

,Gender,Age,EstimatedSalary,Purchased
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0


In [111]:
X = data.drop(columns=["Purchased"])
y = data.Purchased
x_train,x_test,y_train,y_test =  train_test_split(X,y,test_size=0.2)

In [135]:
x_train.head()

,Gender,Age,EstimatedSalary
305,Male,42,54000
187,Female,33,41000
336,Male,58,144000
36,Female,33,28000
382,Female,44,139000


In [149]:
ct1 = ColumnTransformer([
    ("scaling",StandardScaler(),[1,2]),
    ("ohe_gender",OneHotEncoder(handle_unknown="ignore"),[0])
],remainder="passthrough")


ct3 =  RandomForestClassifier()

In [142]:
del pipe

In [150]:
pipe = Pipeline(steps=[
    ("tp1",ct1),
    ("model",ct3)

])

In [151]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('tp1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaling', StandardScaler(),
                                                  [1, 2]),
                                                 ('ohe_gender',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  [0])])),
                ('model', RandomForestClassifier())])

In [152]:
pipe.score(x_test,y_test)

0.9125